`COVID19.CSV`
---
```
Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20
Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,106,152,200,237,297,340,408,480,530,591,665,733,779,830,860,889,910,934,950,962,973,982,986,987,988,989,989,989,989,989,989,990,990,990,990,990,990
Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,91,111,114,139,168,191,212,228,253,274,297,315,326,337,342,352,366,372,375,380,381,387,393,395,396,399,399,399,400,400,410,410,411,413,414,414,418
Chongqing,Mainland China,30.0572,107.874,6,9,27,57,75,110,132,147,182,211,247,300,337,366,389,411,426,428,468,486,505,518,529,537,544,551,553,555,560,567,572,573,575,576,576,576,576,576,576,576,576,576,576
...
```

In [ ]:
# TASK 1.1
import csv
with open('COVID19.CSV') as f:
    c = csv.reader(f)
    header = next(c)
    covid = list(c)

# we note that the date format is M/DD/YY

countries = [i[1] for i in covid]
countires_unique = list(set(countries))
dates = [i.split('/')[2] + i.split('/')[0].zfill(2) + i.split('/')[1].zfill(2) for i in header[4:]]

print(f'Number of countries in data collection: {len(countires_unique)}')
print(f'Number of days in data collection: {len(header[4:])}')
print(f'Start Date of collection (YYMMDD): {sorted(dates)[0]}')
print(f'End Date of collection (YYMMDD): {sorted(dates)[-1]}')

# **[3]**

In [ ]:
# TASK 1.2

country_agg = {}

for country in countires_unique:
    if country not in country_agg.keys():
        country_agg[country] = [0 for _ in range(len(header[4:]))]

for country in country_agg.keys():
    for record in covid:
        if record[1] == country:
            for i in range(len(header[4:])):
                country_agg[country][i] += int(record[i+4])
output = f"Country/Region,{','.join(header[4:])}\n"
                
countires_unique_sorted = sorted(countires_unique)

for country in countires_unique_sorted:
    output += f'{country},{",".join([str(i) for i in country_agg[country]])}\n'
    
with open('countries.csv', 'w') as f:
    f.write(output)

In [ ]:
output

# **[10]**

In [ ]:
# Task 1.3

intraday_max = {}

for country in countires_unique:
    if country not in intraday_max.keys():
        intraday_max[country] = None

for country in intraday_max.keys():
    intraday_max[country] = [country,max(country_agg[country]), header[4:][country_agg[country].index(max(country_agg[country]))]]

ranked = sorted(intraday_max.values(),key=lambda x: x[1], reverse=True)
for i in range(5):
    print(f'{ranked[i][0]:<20}{ranked[i][1]:<20}{ranked[i][2]:<20}')


In [ ]:
sorted(intraday_max.values(),key=lambda x: x[1], reverse=True)

# * Not the maximum cases for each country, but the maximum rise in cases for each day , ie max ( day2 - day1, day3-day2, day_n - day_n-1)
# **[3]**

In [ ]:
# Task 1.4

import pymongo

conn = pymongo.MongoClient('localhost', 27017)
db = conn.get_database('covid')
coll = db.get_collection('cases')

mongo_stuff = []

for idx, date in enumerate(header[4:]):
    to_append = {'date':date,'countries':[]}
    tmp = []
    for country in country_agg.keys():
        tmp.append({'country': country, 'cases': country_agg[country][idx]})
    to_append['countries'] = tmp
    mongo_stuff.append(to_append)
    


coll.insert_many(mongo_stuff)

conn.close()

# **[5]**

In [ ]:
import pymongo

conn = pymongo.MongoClient('localhost', 27017)
db = conn.get_database('covid')
coll = db.get_collection('cases')

print(list(coll.find({})))

conn.close()

In [ ]:
country_agg

In [ ]:
# Task 1.5

import pymongo

conn = pymongo.MongoClient('localhost', 27017)
db = conn.get_database('covid')
coll = db.get_collection('cases')

print(list(coll.find({'date': '1/28/20', 'countries.country':'Mainland China'}))[0])

conn.close()

# * "countries" is an array , not a scalar value
# **[0]**
# **Total :21/25**